In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp

In [ ]:
##Note: If you encounter an error while accessing or writing to the `raw-data` folder in the container, 
# make sure that the IAM role associated with this folder is granted the 'Storage Blob Data Contributor' access. 
# This role is necessary for reading, writing, and deleting blob data in Azure Storage.

configs = {
"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "<Application (client) ID>",
"fs.azure.account.oauth2.client.secret": "<Secret Key>",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<Directory (tenant) ID>/oauth2/token"}

dbutils.fs.mount(
        source="abfss://fifa-world-cup-data@fifaworldcupabhilash.dfs.core.windows.net",
        mount_point="/mnt/fifaworldcup",
        extra_configs=configs
    )

True

In [ ]:
%fs
ls "/mnt/fifaworldcup"

path,name,size,modificationTime
dbfs:/mnt/fifaworldcup/raw-data/,raw-data/,0,1725176572000
dbfs:/mnt/fifaworldcup/transformed-data/,transformed-data/,0,0


In [ ]:
spark

In [ ]:
worldcup = spark.read.format("csv").option("header", "true").load("/mnt/fifaworldcup/raw-data/worldcup.csv")
worldcupmatches = spark.read.format("csv").option("header", "true").load("/mnt/fifaworldcup/raw-data/worldcupmatches.csv")
worldcupplayers = spark.read.format("csv").option("header", "true").load("/mnt/fifaworldcup/raw-data/worldcupplayers.csv")

In [ ]:
worldcup.show(5)

+----+-----------+----------+--------------+-------+----------+-----------+--------------+-------------+----------+
|Year|    Country|    Winner|    Runners-Up|  Third|    Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+-----------+----------+--------------+-------+----------+-----------+--------------+-------------+----------+
|1930|    Uruguay|   Uruguay|     Argentina|    USA|Yugoslavia|         70|            13|           18|   590.549|
|1934|      Italy|     Italy|Czechoslovakia|Germany|   Austria|         70|            16|           17|   363.000|
|1938|     France|     Italy|       Hungary| Brazil|    Sweden|         84|            15|           18|   375.700|
|1950|     Brazil|   Uruguay|        Brazil| Sweden|     Spain|         88|            13|           22| 1.045.246|
|1954|Switzerland|Germany FR|       Hungary|Austria|   Uruguay|        140|            16|           26|   768.607|
+----+-----------+----------+--------------+-------+----------+---------

In [ ]:
worldcup.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Runners-Up: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: string (nullable = true)
 |-- QualifiedTeams: string (nullable = true)
 |-- MatchesPlayed: string (nullable = true)
 |-- Attendance: string (nullable = true)



In [ ]:
worldcup = worldcup.withColumn("GoalsScored", col("GoalsScored").cast("integer")) \
       .withColumn("QualifiedTeams", col("QualifiedTeams").cast("integer")) \
        .withColumn("Year", col("Year").cast("integer")) \
       .withColumn("MatchesPlayed", col("MatchesPlayed").cast("integer")) \
       .withColumn("Attendance", col("Attendance").cast("integer"))

In [ ]:
worldcup.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Runners-Up: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: integer (nullable = true)
 |-- QualifiedTeams: integer (nullable = true)
 |-- MatchesPlayed: integer (nullable = true)
 |-- Attendance: integer (nullable = true)



In [ ]:
df_goals_by_country = worldcup.groupBy("Country").sum("GoalsScored").withColumnRenamed("sum(GoalsScored)", "TotalGoals").orderBy(col("TotalGoals").desc()).show()


+------------+----------+
|     Country|TotalGoals|
+------------+----------+
|      Brazil|       259|
|      France|       255|
|     Germany|       244|
|      Mexico|       227|
|       Italy|       185|
| Korea/Japan|       161|
|       Spain|       146|
|South Africa|       145|
|         USA|       141|
| Switzerland|       140|
|      Sweden|       126|
|   Argentina|       102|
|     England|        89|
|       Chile|        89|
|     Uruguay|        70|
+------------+----------+



In [ ]:
worldcup_winners_count = worldcup.groupBy("Winner").count().withColumnRenamed("count", "NumberOfWins")
top_3_countries = worldcup_winners_count.orderBy(col("NumberOfWins").desc()).limit(3)
top_3_countries.show()

+----------+------------+
|    Winner|NumberOfWins|
+----------+------------+
|    Brazil|           5|
|     Italy|           4|
|Germany FR|           3|
+----------+------------+



In [ ]:
worldcupmatches.show(5)

+----+--------------------+-------+--------------+-----------+--------------+---------------+---------------+--------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|  Stage|       Stadium|       City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+----+--------------------+-------+--------------+-----------+--------------+---------------+---------------+--------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|1930|13 Jul 1930 - 15:00 |Group 1|       Pocitos|Montevideo |        

In [ ]:
worldcupmatches.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Home Team Name: string (nullable = true)
 |-- Home Team Goals: string (nullable = true)
 |-- Away Team Goals: string (nullable = true)
 |-- Away Team Name: string (nullable = true)
 |-- Win conditions: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- Half-time Home Goals: string (nullable = true)
 |-- Half-time Away Goals: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Assistant 1: string (nullable = true)
 |-- Assistant 2: string (nullable = true)
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Home Team Initials: string (nullable = true)
 |-- Away Team Initials: string (nullable = true)



In [ ]:
worldcupmatches = worldcupmatches.withColumn("Home Team Goals", col("Home Team Goals").cast("integer")) \
        .withColumn("Year", col("Year").cast("integer")) \
       .withColumn("Away Team Goals", col("Away Team Goals").cast("integer")) \
       .withColumn("Attendance", col("Attendance").cast("integer")) \
       .withColumn("Half-time Home Goals", col("Half-time Home Goals").cast("integer")) \
       .withColumn("Half-time Away Goals", col("Half-time Away Goals").cast("integer"))

In [ ]:
worldcupmatches.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Datetime: timestamp (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Home Team Name: string (nullable = true)
 |-- Home Team Goals: integer (nullable = true)
 |-- Away Team Goals: integer (nullable = true)
 |-- Away Team Name: string (nullable = true)
 |-- Win conditions: string (nullable = true)
 |-- Attendance: integer (nullable = true)
 |-- Half-time Home Goals: integer (nullable = true)
 |-- Half-time Away Goals: integer (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Assistant 1: string (nullable = true)
 |-- Assistant 2: string (nullable = true)
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Home Team Initials: string (nullable = true)
 |-- Away Team Initials: string (nullable = true)



In [ ]:
worldcupmatches = worldcupmatches.withColumn("Datetime", to_timestamp(col("Datetime"), "dd/MM/yyyy HH:mm"))

In [ ]:
home_goals = worldcupmatches.groupBy("Home Team Name").sum("Home Team Goals").withColumnRenamed("Home Team Name", "Team Name").withColumnRenamed("sum(Home Team Goals)", "Goals")
away_goals = worldcupmatches.groupBy("Away Team Name").sum("Away Team Goals").withColumnRenamed("Away Team Name", "Team Name").withColumnRenamed("sum(Away Team Goals)", "Goals")

combined_goals = home_goals.unionByName(away_goals)
df_total_goals = combined_goals.groupBy("Team Name").sum("Goals").withColumnRenamed("sum(Goals)", "TotalGoals").orderBy(col("TotalGoals").desc())

df_total_goals.show()

+--------------+----------+
|     Team Name|TotalGoals|
+--------------+----------+
|        Brazil|       225|
|     Argentina|       133|
|    Germany FR|       131|
|         Italy|       128|
|        France|       108|
|       Germany|       104|
|         Spain|        92|
|   Netherlands|        91|
|       Hungary|        87|
|       Uruguay|        80|
|       England|        79|
|        Sweden|        74|
|    Yugoslavia|        60|
|        Mexico|        58|
|       Belgium|        54|
|  Soviet Union|        53|
|   Switzerland|        45|
|Czechoslovakia|        44|
|        Poland|        44|
|      Portugal|        43|
+--------------+----------+
only showing top 20 rows



In [ ]:
worldcupplayers.show(5)

+-------+-------+-------------+-------------------+-------+------------+----------------+--------+-----+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|     Player Name|Position|Event|
+-------+-------+-------------+-------------------+-------+------------+----------------+--------+-----+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|     Alex THEPOT|      GK| NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0| Oscar BONFIGLIO|      GK| NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|Marcel LANGILLER|    NULL| G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|    Juan CARRENO|    NULL| G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Ernest LIBERATI|    NULL| NULL|
+-------+-------+-------------+-------------------+-------+------------+----------------+--------+-----+
only showing top 5 rows



In [ ]:
worldcupplayers.printSchema()

root
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: string (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
worldcupplayers = worldcupplayers.withColumn("RoundID", col("RoundID").cast("integer")) \
       .withColumn("MatchID", col("MatchID").cast("integer")) \
       .withColumn("Shirt Number", col("Shirt Number").cast("integer")) \
       .withColumn("Shirt Number", col("Shirt Number").cast("integer"))


In [ ]:
worldcupplayers.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
worldcup.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/fifaworldcup/transformed-data/worldcup")

In [ ]:
worldcupmatches.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/fifaworldcup/transformed-data/worldcupmatches")
worldcupplayers.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/fifaworldcup/transformed-data/worldcupplayers")
